In [1]:
from mcrcon import MCRcon
import time

from redstone import RedstoneFactory

PLAYER_NAME = "Mihnea8848"

def load_and_paste():
    with MCRcon(host="localhost", password="test") as mcrcon:
        mcrcon.command("sudo Mihnea8848 /undo")
        mcrcon.command("sudo Mihnea8848 /schem load devschematics/testt")
        mcrcon.command("sudo Mihnea8848 /paste -a")


INFO - PyMCTranslate Version 377


In [2]:
import importlib
from redstone import RedstoneFactory, AmuletSchematic
from yosys_parser import parse_yosys_json
from Optimizer import Optimizer

In [3]:
factory = RedstoneFactory()

nodes = parse_yosys_json("counter.json", factory)
optimizer = Optimizer(factory, bounds=(70, 70),startT=30000,endT=0.01,cooling_rate=0.995)

Parsing module: counter
  Port Input clk (Net 2) -> Node 0
  Port Input rst (Net 3) -> Node 1
Linking graph...
Successfully created 27 components and 42 connections.


In [4]:
optimizer.run(steps=50000)


Starting SA. Initial Energy: 353954
  >>> New Best at Step 0: 351950
Step 0/50000 | Temp: 30000.00 | Current: 351950 | Best: 351950
  >>> New Best at Step 1: 350476
  >>> New Best at Step 3: 350472
  >>> New Best at Step 4: 306786
  >>> New Best at Step 5: 306782
  >>> New Best at Step 9: 253406
  >>> New Best at Step 12: 234808
  >>> New Best at Step 13: 230602
  >>> New Best at Step 14: 229608
  >>> New Best at Step 19: 228604
Step 100/50000 | Temp: 18173.11 | Current: 284081 | Best: 228604
Step 200/50000 | Temp: 11008.73 | Current: 272198 | Best: 228604
  >>> New Best at Step 254: 226179
  >>> New Best at Step 259: 198947
  >>> New Best at Step 260: 197943
  >>> New Best at Step 269: 194940
  >>> New Best at Step 270: 193940
  >>> New Best at Step 283: 191926
Step 300/50000 | Temp: 6668.77 | Current: 216014 | Best: 191926
Step 400/50000 | Temp: 4039.74 | Current: 205878 | Best: 191926
  >>> New Best at Step 403: 183914
  >>> New Best at Step 405: 182904
  >>> New Best at Step 430: 1

45054

In [5]:
schematic = AmuletSchematic()
factory.compile_to_schematic(schematic)

Compiling 27 components...
Calculating 3D Obstacle Grid...
Tracing underground clock...
Routing Logic Signals (3D Mode with Fan-Out & Observer Towers)...


In [6]:
schematic.save("devschematics", "tedx")

INFO - Saving changes to level devschematics/tedx.schematic
INFO - Finished saving changes to level devschematics/tedx.schematic


Saved to: devschematics/tedx.schematic


In [7]:
import multiprocessing
import traceback
import random
# Ensure these are saved in .py files next to your notebook

# --- ROBUST WORKER FUNCTION ---
def run_optimization_safe(seed):
    """
    Wrapper that catches exceptions and returns them as data.
    """
    try:
        # 1. Setup
        random.seed(seed)
        factory = RedstoneFactory()

        # Ensure counter.json exists or pass absolute path
        nodes = parse_yosys_json("counter.json", factory)
        if not nodes:
            return None, 0, "Parser returned 0 nodes"

        # 2. Run
        # Using fewer steps for testing validity first
        opt = Optimizer(factory, bounds=(100, 100), startT=5000, endT=0.1)
        best_score = opt.run(steps=2000)

        return factory, best_score, "OK"

    except Exception:
        # Capture the full crash report
        error_msg = traceback.format_exc()
        return None, float('inf'), error_msg

# --- MAIN EXECUTION ---
if __name__ == "__main__":
    NUM_CHAINS = multiprocessing.cpu_count()
    print(f"Launching {NUM_CHAINS} parallel optimization chains...")

    with multiprocessing.Pool(NUM_CHAINS) as pool:
        # Map returns a list of tuples: [(factory, score, status), ...]
        raw_results = pool.map(run_optimization_safe, range(NUM_CHAINS))

    # 1. SEPARATE SUCCESSES FROM FAILURES
    successes = []
    failures = []

    for res in raw_results:
        if res and res[2] == "OK":
            successes.append(res)
        else:
            failures.append(res)

    # 2. REPORT ERRORS IF ANY
    if failures:
        print(f"\nWarning: {len(failures)} chains failed.")
        print("Sample Error from first failure:")
        # Print the traceback captured inside the worker
        print(failures[0][2])

    # 3. PROCESS WINNER
    if successes:
        # Find minimum score (index 1 is score)
        best_factory, best_score, _ = min(successes, key=lambda x: x[1])

        print(f"\nWinner Energy: {best_score}")

        schem = AmuletSchematic()
        best_factory.compile_to_schematic(schem)
        schem.save("devschematics", "final")
        print("Saved successfully.")
    else:
        print("\nCRITICAL: All chains failed. See error message above.")

Launching 20 parallel optimization chains...
Parsing module: counterParsing module: counterParsing module: counterParsing module: counterParsing module: counterParsing module: counterParsing module: counterParsing module: counterParsing module: counterParsing module: counterParsing module: counterParsing module: counterParsing module: counterParsing module: counterParsing module: counterParsing module: counterParsing module: counter
Parsing module: counter






Parsing module: counter
Parsing module: counter






  Port Input clk (Net 2) -> Node 0
  Port Input clk (Net 2) -> Node 0  Port Input clk (Net 2) -> Node 0
  Port Input clk (Net 2) -> Node 0  Port Input clk (Net 2) -> Node 0  Port Input clk (Net 2) -> Node 0  Port Input clk (Net 2) -> Node 0  Port Input clk (Net 2) -> Node 0  Port Input clk (Net 2) -> Node 0
  Port Input clk (Net 2) -> Node 0
  Port Input clk (Net 2) -> Node 0  Port Input clk (Net 2) -> Node 0  Port Input clk (Net 2) -> Node 0  Port Input clk (Net 2) -> Node 

INFO - Saving changes to level devschematics/final.schematic
INFO - Finished saving changes to level devschematics/final.schematic


Saved to: devschematics/final.schematic
Saved successfully.


In [8]:
import multiprocessing
import traceback
import random
# Ensure these are saved in .py files next to your notebook

# --- ROBUST WORKER FUNCTION ---
def run_optimization_safe(seed):
    """
    Wrapper that catches exceptions and returns them as data.
    """
    try:
        # 1. Setup
        random.seed(seed)
        factory = RedstoneFactory()

        # Ensure counter.json exists or pass absolute path
        nodes = parse_yosys_json("counter.json", factory)
        if not nodes:
            return None, 0, "Parser returned 0 nodes"

        # 2. Run
        # Using fewer steps for testing validity first
        opt = Optimizer(factory, bounds=(100, 100), startT=5000, endT=0.1)
        best_score = opt.run(steps=2000)

        return factory, best_score, "OK"

    except Exception:
        # Capture the full crash report
        error_msg = traceback.format_exc()
        return None, float('inf'), error_msg

# --- MAIN EXECUTION ---
if __name__ == "__main__":
    NUM_CHAINS = multiprocessing.cpu_count()
    print(f"Launching {NUM_CHAINS} parallel optimization chains...")

    with multiprocessing.Pool(NUM_CHAINS) as pool:
        # Map returns a list of tuples: [(factory, score, status), ...]
        raw_results = pool.map(run_optimization_safe, range(NUM_CHAINS))

    # 1. SEPARATE SUCCESSES FROM FAILURES
    successes = []
    failures = []

    for res in raw_results:
        if res and res[2] == "OK":
            successes.append(res)
        else:
            failures.append(res)

    # 2. REPORT ERRORS IF ANY
    if failures:
        print(f"\nWarning: {len(failures)} chains failed.")
        print("Sample Error from first failure:")
        # Print the traceback captured inside the worker
        print(failures[0][2])

    # 3. PROCESS WINNER
    if successes:
        # Find minimum score (index 1 is score)
        best_factory, best_score, _ = min(successes, key=lambda x: x[1])

        print(f"\nWinner Energy: {best_score}")

        schem = AmuletSchematic()
        best_factory.compile_to_schematic(schem)
        schem.save("devschematics", "final")
        print("Saved successfully.")
    else:
        print("\nCRITICAL: All chains failed. See error message above.")

Launching 20 parallel optimization chains...
Parsing module: counterParsing module: counterParsing module: counterParsing module: counterParsing module: counterParsing module: counterParsing module: counterParsing module: counterParsing module: counterParsing module: counter
Parsing module: counterParsing module: counterParsing module: counterParsing module: counterParsing module: counter
Parsing module: counter



Parsing module: counter

  Port Input clk (Net 2) -> Node 0





Parsing module: counter  Port Input clk (Net 2) -> Node 0  Port Input clk (Net 2) -> Node 0
  Port Input clk (Net 2) -> Node 0  Port Input clk (Net 2) -> Node 0  Port Input clk (Net 2) -> Node 0
  Port Input clk (Net 2) -> Node 0  Port Input clk (Net 2) -> Node 0  Port Input clk (Net 2) -> Node 0
  Port Input clk (Net 2) -> Node 0  Port Input clk (Net 2) -> Node 0  Port Input clk (Net 2) -> Node 0  Port Input clk (Net 2) -> Node 0


  Port Input clk (Net 2) -> Node 0

  Port Input clk (Net 2) -> Node 0

  Port 

INFO - Saving changes to level devschematics/final.schematic
INFO - Finished saving changes to level devschematics/final.schematic


Saved to: devschematics/final.schematic
Saved successfully.


In [9]:
print(f"Total Nodes: {len(factory.graph.nodes)}")

node_counts = {}
for node in factory.graph.nodes:
    type_name = node.type
    node_counts[type_name] = node_counts.get(type_name, 0) + 1

    # If we find a WIRE or INPUT, print its location
    if type_name in ["WIRE", "INPUT"]:
        print(f"Found {type_name} at {node.pos}")

print("\n--- Node Type Counts ---")
print(node_counts)

Total Nodes: 27
Found INPUT at (72, 4, 64)
Found INPUT at (58, 4, 110)

--- Node Type Counts ---
{'INPUT': 2, 'NOT': 8, 'NOR': 13, 'DFF': 4}


In [10]:
visualize_circuit(factory.graph)

NameError: name 'visualize_circuit' is not defined

In [11]:
load_and_paste()